In [3]:
from datasets import load_dataset

datasets = load_dataset("glue", "mrpc")

print(datasets["train"].features)

Found cached dataset glue (/home/vscode/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 736.06it/s]

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None), 'idx': Value(dtype='int32', id=None)}


In [4]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer("what is going on")

In [4]:
from datasets import load_dataset

from transformers import AutoTokenizer,  AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets)
print(raw_datasets['train'].features)

# As long as tokenizer and model are using same checkpoint, everything will be ok.
checkpoint = "bert-base-uncased"

# Convert string to integers
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Allows us to use map(batched=True) which speeds up tokenization.
def tokenize_function(example):
    # Leave out `padding` b/c more efficient to pad to maximum size of batch in DataCollator
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Tokenize datastes
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# tokenizer arg used to get: padding token, whether to pad on left or right.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Contains all hyperparameters the Trainer will use for training and evaluation.
training_args = TrainingArguments("test-trainer")

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    # data_collator=data_collator, default is DataCollatorWithPadding with `tokenizer` arg passed.
    tokenizer=tokenizer,
)

trainer.train()

Step,Training Loss
500,0.494100
